<a href="https://colab.research.google.com/github/fenzhantw/Dacon_jobcare_recommend/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모듈 불러오기

In [ ]:
!pip install catboost

In [ ]:
!pip install optuna

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import f1_score 
from sklearn.feature_extraction import FeatureHasher

from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier 
from catboost import Pool,CatBoostClassifier

##함수화 작업

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/job_care/'


#분류 추가
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df
 

def drop_features(df):
  df = df.drop(['id','person_rn','contents_rn','person_prefer_f','person_prefer_g'],axis=1)
  return df

def get_date(df):
  for i in range(2):
    df[i].contents_open_dt = pd.to_datetime(df[i].contents_open_dt)

    df[i]['month'] = df[i].contents_open_dt.dt.month
    df[i]['day'] = df[i].contents_open_dt.dt.day
    df[i]['week'] = df[i].contents_open_dt.dt.isocalendar().week
    df[i]['dayofweek'] = df[i].contents_open_dt.dt.dayofweek
    df[i]['hour'] = df[i].contents_open_dt.dt.hour
  #  df[i]['minute'] = df[i].contents_open_dt.dt.minute

    df[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

def show_hist_by_target(df,columns):
  cond_1 = (df['target'] == 1)
  cond_0 = (df['target'] == 0)

  for column in columns:
    fix,axs = plt.subplots(nrows=1,ncols=2,figsize = (12, 4),squeeze=False)
    sns.violinplot(x='target',y=column,data=df,ax=axs[0][0])
    sns.distplot(df[cond_0][column],ax=axs[0][1],label='0',color='blue')
    sns.distplot(df[cond_1][column],ax=axs[0][1],label='1',color='red')

# 10개 이상 카테고리 얻기
def get_data_up10(df):
  up10_cat = []
  columns_names=df.columns.values
  for i in columns_names:
    if df[i].value_counts().count() >=10:
      up10_cat.append(i)

  up10_cat.append('target')
  up10_cat.remove('contents_open_dt')

  df2 = df[up10_cat]
  train_target_1 = df[df['target']==1]
  train_target_0 = df[df['target']==0]
  
  return train_target_1,train_target_0,up10_cat

# 데이터 비율 차이 구하기
def get_data_dict(train_target_1,train_target_0,up10_cat):
  list_diff=[]
  for i in up10_cat[:-1]:
    serise=abs((train_target_1[i].value_counts()/train_target_1[i].shape[0])*100 - (train_target_0[i].value_counts()/train_target_0[i].shape[0])*100)
    temp_list=serise[serise>=0.5].index.tolist()
    list_diff.append(temp_list)

  dict1 = {up10_cat[i]:list_diff[i] for i in range(len(up10_cat[:-1])) if len(list_diff[i])>1 }
  return dict1

# 데이터 비율의 차이가 0.5보다 크면 1 아니면 0
def get_data_1_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_{key}"] =1
  return train,test

# 기존의 0.5이상인 컬럼 값은 들고오고,이하인것만 0으로 된 컬럼 추가
def get_data_data_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_div_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_div_{key}"] = i  
  return train,test

# 10개 이하 카테고리 얻기

def get_data_down10(df):
  col_name = list(df.columns)
  object_nunique = list(map(lambda col: df[col].nunique(), col_name))
  d = dict(zip(col_name, object_nunique))
  d = pd.DataFrame([col_name, object_nunique]).T
  onehot_list = list(d[(d[1]>2)&(d[1]<=10)][0])

  return onehot_list

# 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.5이상이면 1 아니면 0인 변수 추가/ 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.55이상이면 1, 0.45~0.55사이면 2, 0.45 이하이면 3인 변수 추가

def get_down10_1_or_0(train,test,onehot_list):
  for col in onehot_list:
    temp_df = []
        
    # 명목형 변수에서 각 값 별로 타겟값의 평균을 대입
    feat = train.groupby(col)["target"].agg("mean")
    feat = feat.to_dict()
    test.loc[:, f"tar_enc_{col}"] = X_test[col].map(feat)
    temp_df.append(test)

    temp_train_feat = train[col].map(feat)
    temp_test_feat = test[col].map(feat)

    train.loc[:, f"tar_enc_{col}"] = temp_train_feat
    test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    train.drop('target', axis=1, inplace=True)

    return train,test,temp_df

def div_value_2(temp):
    new_value = ''

    if temp >=0.5 : new_value = 1
    else : new_value = 0
    
    return new_value


def div_value_3(temp):
    new_value = ''

    if temp >=0.55 : new_value = 1
    elif temp <=0.45 : new_value = -1
    else : new_value = 0
    
    return new_value

def get_down10_1_or_00(train,X_train,test,onehot_list):
    div_value = list(train.iloc[:,X_train.shape[1]:X_train.shape[1]+abs(X_train.shape[1] - train.shape[1])].columns)

    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div2_{onehot_list[i]}"] = train.loc[:,div_value[i]].apply(lambda x : div_value_2(x))
      test.loc[:, f"tar_div2_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_2(x))
      
    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div3_{onehot_list[i]}"] = train.loc[:, div_value[i]].apply(lambda x : div_value_3(x))
      test.loc[:, f"tar_div3_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_3(x))

    return X_train,test



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def drop_features(df):
  df = df.drop(['id','person_rn','contents_rn','person_prefer_f','person_prefer_g'],axis=1)
  return df

def get_date(df):
  for i in range(2):
    df[i].contents_open_dt = pd.to_datetime(df[i].contents_open_dt)

    df[i]['month'] = df[i].contents_open_dt.dt.month
    df[i]['day'] = df[i].contents_open_dt.dt.day
    df[i]['week'] = df[i].contents_open_dt.dt.isocalendar().week
    df[i]['dayofweek'] = df[i].contents_open_dt.dt.dayofweek
    df[i]['hour'] = df[i].contents_open_dt.dt.hour
  #  df[i]['minute'] = df[i].contents_open_dt.dt.minute

    df[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

def show_hist_by_target(df,columns):
  cond_1 = (df['target'] == 1)
  cond_0 = (df['target'] == 0)

  for column in columns:
    fix,axs = plt.subplots(nrows=1,ncols=2,figsize = (12, 4),squeeze=False)
    sns.violinplot(x='target',y=column,data=df,ax=axs[0][0])
    sns.distplot(df[cond_0][column],ax=axs[0][1],label='0',color='blue')
    sns.distplot(df[cond_1][column],ax=axs[0][1],label='1',color='red')


In [ ]:
  train = pd.read_csv(path + "train.csv")
  test = pd.read_csv(path + "test.csv")
  submission = pd.read_csv(path + "sample_submission.csv")
  d_code = pd.read_csv(path + '속성_D_코드.csv', index_col=0).T.to_dict()
  h_code = pd.read_csv(path + '속성_H_코드.csv', index_col=0).T.to_dict()
  l_code = pd.read_csv(path + '속성_L_코드.csv', index_col=0).T.to_dict()

In [ ]:
# 10개 이상 카테고리 얻기
def get_data_up10(df):
  up10_cat = []
  columns_names=df.columns.values
  for i in columns_names:
    if df[i].value_counts().count() >=10:
      up10_cat.append(i)

  up10_cat.append('target')
  up10_cat.remove('contents_open_dt')

  df2 = df[up10_cat]
  train_target_1 = df[df['target']==1]
  train_target_0 = df[df['target']==0]
  
  return train_target_1,train_target_0,up10_cat

# 데이터 비율 차이 구하기
def get_data_dict(train_target_1,train_target_0,up10_cat):
  list_diff=[]
  for i in up10_cat[:-1]:
    serise=abs((train_target_1[i].value_counts()/train_target_1[i].shape[0])*100 - (train_target_0[i].value_counts()/train_target_0[i].shape[0])*100)
    temp_list=serise[serise>=0.5].index.tolist()
    list_diff.append(temp_list)

  dict1 = {up10_cat[i]:list_diff[i] for i in range(len(up10_cat[:-1])) if len(list_diff[i])>1 }
  return dict1

# 데이터 비율의 차이가 0.5보다 크면 1 아니면 0
def get_data_1_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_{key}"] =1
  return train,test

# 기존의 0.5이상인 컬럼 값은 들고오고,이하인것만 0으로 된 컬럼 추가
def get_data_data_or_0(train,test,dict1):
  for data in [train,test]:
    for key,values in dict1.items():
      data.loc[:, f"tar_enc_div_{key}"] = 0
      for i in values:
        con=data[data[key]==i].index
        data.loc[con, f"tar_enc_div_{key}"] = i  
  return train,test


In [ ]:
# 10개 이하 카테고리 얻기

def get_data_down10(df):
  col_name = list(df.columns)
  object_nunique = list(map(lambda col: df[col].nunique(), col_name))
  d = dict(zip(col_name, object_nunique))
  d = pd.DataFrame([col_name, object_nunique]).T
  onehot_list = list(d[(d[1]>2)&(d[1]<=10)][0])

  return onehot_list

# 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.5이상이면 1 아니면 0인 변수 추가/ 범주 갯수가 10개 이하인 변수에 대해서 타겟값이 0.55이상이면 1, 0.45~0.55사이면 2, 0.45 이하이면 3인 변수 추가

def get_down10_1_or_0(train,test,onehot_list):
  for col in onehot_list:
    temp_df = []
        
    # 명목형 변수에서 각 값 별로 타겟값의 평균을 대입
    feat = train.groupby(col)["target"].agg("mean")
    feat = feat.to_dict()
    test.loc[:, f"tar_enc_{col}"] = X_test[col].map(feat)
    temp_df.append(test)

    temp_train_feat = train[col].map(feat)
    temp_test_feat = test[col].map(feat)

    train.loc[:, f"tar_enc_{col}"] = temp_train_feat
    test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    train.drop('target', axis=1, inplace=True)

    return train,test,temp_df

def div_value_2(temp):
    new_value = ''

    if temp >=0.5 : new_value = 1
    else : new_value = 0
    
    return new_value


def div_value_3(temp):
    new_value = ''

    if temp >=0.55 : new_value = 1
    elif temp <=0.45 : new_value = -1
    else : new_value = 0
    
    return new_value

def get_down10_1_or_00(train,X_train,test,onehot_list):
    div_value = list(train.iloc[:,X_train.shape[1]:X_train.shape[1]+abs(X_train.shape[1] - train.shape[1])].columns)

    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div2_{onehot_list[i]}"] = train.loc[:,div_value[i]].apply(lambda x : div_value_2(x))
      test.loc[:, f"tar_div2_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_2(x))
      
    for i in range(len(div_value)):
      X_train.loc[:, f"tar_div3_{onehot_list[i]}"] = train.loc[:, div_value[i]].apply(lambda x : div_value_3(x))
      test.loc[:, f"tar_div3_{onehot_list[i]}"] = test.loc[:, div_value[i]].apply(lambda x : div_value_3(x))

    return X_train,test


In [ ]:
train = add_code(train, d_code, h_code, l_code)
train = drop_features(train)
test = add_code(test, d_code, h_code, l_code)
test = drop_features(test)
y_train = train.loc[:,'target']
X_train = train.drop(['target'],axis=1)
train_target_1,train_target_0,up10_cat = get_data_up10(train)
dict1 = get_data_dict(train_target_1,train_target_0,up10_cat)
train,test = get_data_1_or_0(train,test,dict1)
train,test = get_data_data_or_0(train,test,dict1)


## 시작

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/job_care/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
train = pd.read_csv(path + "train.csv")
X_test = pd.read_csv(path + "test.csv")
submission = pd.read_csv(path + "sample_submission.csv")

d_code = pd.read_csv(path + '속성_D_코드.csv', index_col=0)
h_code = pd.read_csv(path + '속성_H_코드.csv', index_col=0)
l_code = pd.read_csv(path + '속성_L_코드.csv', index_col=0)

In [ ]:
d_code = d_code.T.to_dict()
h_code = h_code.T.to_dict()
l_code = l_code.T.to_dict()

In [ ]:
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df

train = add_code(train, d_code, h_code, l_code)
X_test = add_code(X_test, d_code, h_code, l_code)
print("train_data.shape: ", train.shape)
print("test_data.shape: ", X_test.shape)

train_data.shape:  (501951, 63)
test_data.shape:  (46404, 62)


## 컬럼 추가

In [ ]:
data = [train, X_test]

for i in range(2):
  data[i].contents_open_dt = pd.to_datetime(data[i].contents_open_dt)

  data[i]['month'] = data[i].contents_open_dt.dt.month
  data[i]['day'] = data[i].contents_open_dt.dt.day
  data[i]['week'] = data[i].contents_open_dt.dt.isocalendar().week
  data[i]['dayofweek'] = data[i].contents_open_dt.dt.dayofweek
  data[i]['hour'] = data[i].contents_open_dt.dt.hour
#  data[i]['minute'] = data[i].contents_open_dt.dt.minute

  data[i].drop(['id', 'contents_open_dt'], axis=1, inplace=True)

In [ ]:
X_train = train.drop('target',axis=1)
y_train = train['target']

In [ ]:
X_train = X_train.drop(['person_rn', 'contents_rn', 'person_prefer_f', 'person_prefer_g'],axis=1)
X_test = X_test.drop(['person_rn', 'contents_rn', 'person_prefer_f', 'person_prefer_g'],axis=1)

In [ ]:
col_name = list(X_train.columns)

In [ ]:
object_nunique = list(map(lambda col: X_train[col].nunique(), col_name))
d = dict(zip(col_name, object_nunique))

# # Print number of unique entries by column, in ascending order
# sorted(d.items(), key=lambda x: x[1])

In [ ]:
d = pd.DataFrame([col_name, object_nunique]).T
onehot_list = list(d[(d[1]>2)&(d[1]<=10)][0])

In [ ]:
df = pd.concat([X_train, y_train], axis=1)
df_test = X_test.copy()

for col in onehot_list:
    # 명목형 변수에서 각 값 별로 타겟값의 평균을 대입
    feat = df.groupby(col)["target"].agg("mean")
    feat = feat.to_dict()

    # 명목형 변수에 매칭되는 각 평균값을 temp_train/test_feat로 생성
    temp_train_feat = df[col].map(feat)
    temp_test_feat = df_test[col].map(feat)

    #temp_train/test_feat를 tar_enc_{col} 형태로 컬럼 생성
    df.loc[:, f"tar_enc_{col}"] = temp_train_feat
    df_test.loc[:, f"tar_enc_{col}"] = temp_test_feat

# df.drop('target', axis=1, inplace=True)

In [ ]:
#분기 함수 생성
def div_value_2(temp):
    new_value = ''

    if temp >=0.5 : new_value = 1
    else : new_value = 0
    
    return new_value


def div_value_3(temp):
    new_value = ''

    if temp >=0.55 : new_value = 1
    elif temp <=0.45 : new_value = -1
    else : new_value = 0
    
    return new_value

In [ ]:
#위에서 생성된 새로운 컬럼 div_value에 저장
div_value = list(df.iloc[:,X_train.shape[1]:X_train.shape[1]+abs(X_train.shape[1] - df.shape[1])].columns)

#lambda 함수로 위에 생성한 분기값 입력(div_value_2)
for i in range(len(div_value)):
  X_train.loc[:, f"tar_div2_{onehot_list[i]}"] = df.loc[:,div_value[i]].apply(lambda x : div_value_2(x))
  X_test.loc[:, f"tar_div2_{onehot_list[i]}"] = df_test.loc[:, div_value[i]].apply(lambda x : div_value_2(x))

#lambda 함수로 위에 생성한 분기값 입력(div_value_3)
for i in range(len(div_value)):
  X_train.loc[:, f"tar_div3_{onehot_list[i]}"] = df.loc[:, div_value[i]].apply(lambda x : div_value_3(x))
  X_test.loc[:, f"tar_div3_{onehot_list[i]}"] = df_test.loc[:, div_value[i]].apply(lambda x : div_value_3(x))

In [ ]:
# 하나의 값만 가지는 컬럼 drop
drop_feature = X_train.columns[X_train.nunique() < 2].tolist()

X_train.drop(drop_feature,axis=1, inplace=True)
X_test.drop(drop_feature,axis=1, inplace=True)

In [ ]:
# 10개 이상 카테고리 출력
up10_cat = list(d[d[1]>10][0])
up10_cat

['person_prefer_d_1',
 'person_prefer_d_2',
 'person_prefer_d_3',
 'person_prefer_e',
 'person_prefer_h_1',
 'person_prefer_h_2',
 'person_prefer_h_3',
 'contents_attribute_l',
 'contents_attribute_d',
 'contents_attribute_e',
 'contents_attribute_h',
 'person_prefer_d_1_n',
 'person_prefer_d_1_s',
 'person_prefer_d_1_m',
 'person_prefer_d_1_l',
 'person_prefer_d_2_n',
 'person_prefer_d_2_s',
 'person_prefer_d_2_m',
 'person_prefer_d_2_l',
 'person_prefer_d_3_n',
 'person_prefer_d_3_s',
 'person_prefer_d_3_m',
 'person_prefer_d_3_l',
 'contents_attribute_d_n',
 'contents_attribute_d_s',
 'contents_attribute_d_m',
 'contents_attribute_d_l',
 'person_prefer_h_1_m',
 'person_prefer_h_2_m',
 'person_prefer_h_3_m',
 'contents_attribute_h_m',
 'person_prefer_h_1_l',
 'person_prefer_h_2_l',
 'person_prefer_h_3_l',
 'contents_attribute_h_l',
 'contents_attribute_l_n',
 'contents_attribute_l_s',
 'contents_attribute_l_m',
 'contents_attribute_l_l',
 'month',
 'day',
 'week',
 'hour']

In [ ]:
if onehot_list in  up10_cat:
  print(True)
else: print(False)

False


In [ ]:
train_up = df[up10_cat]
test_up  = X_test[up10_cat]

train_up['target'] = y_train

In [ ]:
train_1 = train_up[train_up['target']==1]
train_0 = train_up[train_up['target']==0]

In [ ]:
list_diff=[]
for i in up10_cat[:-1]:
  serise=abs((train_1[i].value_counts()/train_1[i].shape[0])*100 - (train_0[i].value_counts()/train_0[i].shape[0])*100)
  temp_list=serise[serise>=0.5].index.tolist()
  list_diff.append(temp_list)

# 0.5차이 나는 값이 1개 이상인 카테고리 딕셔너리로 담기 / 0.5 차이가 나는 변수가 하나인 컬럼은 dict에서 제외
dict1 = {up10_cat[i]:list_diff[i] for i in range(len(up10_cat[:-1])) if len(list_diff[i])>1 }

In [ ]:
# 컬럼 추가
for data in [train_up,test_up]:
  for key,values in dict1.items():
    data.loc[:, f"tar_enc_1_10_div_{key}"] = 0
    for i in values:
      con=data[data[key]==i].index
      data.loc[con, f"tar_enc_1_10_div_{key}"] = 1

In [ ]:
#기존의 0.5이상인 컬럼 값은 들고오고, 이하인것만 0으로 된 컬럼 추가
for data in [train_up,test_up]:
  for key,values in dict1.items():
    data.loc[:, f"tar_enc_2_10_div_{key}"] = 0
    for i in values:
      con=data[data[key]==i].index
      data.loc[con, f"tar_enc_2_10_div_{key}"] = i

In [ ]:
X_train = pd.concat([X_train, train_up.iloc[:,44:]], axis=1)
X_test = pd.concat([X_test, test_up.iloc[:,43:]], axis=1)

##비지도 학습으로 그룹 변수 추가

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42).fit(X_train)
X_train['cluster_5'] = kmeans.predict(X_train)
X_test['cluster_5'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=42).fit(X_train)
X_train['cluster_10'] = kmeans.predict(X_train)
X_test['cluster_10'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42).fit(X_train)
X_train['cluster_20'] = kmeans.predict(X_train)
X_test['cluster_20'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=30, random_state=42).fit(X_train)
X_train['cluster_30'] = kmeans.predict(X_train)
X_test['cluster_30'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=40, random_state=42).fit(X_train)
X_train['cluster_40'] = kmeans.predict(X_train)
X_test['cluster_40'] = kmeans.predict(X_test)

In [ ]:
kmeans = KMeans(n_clusters=50, random_state=42).fit(X_train)
X_train['cluster_50'] = kmeans.predict(X_train)
X_test['cluster_50'] = kmeans.predict(X_test)

## 최종 데이터셋 출력

In [ ]:
X_train.to_csv(path + "X_train_last.csv", index=False)
X_test.to_csv(path + "X_test_last.csv", index=False)

In [5]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/MyDrive/Colab Notebooks/job_care/'

X_train = pd.read_csv(path + "X_train_last.csv")
X_test = pd.read_csv(path + "X_test_last.csv")
train = pd.read_csv(path + "train.csv")
y_train = train["target"]
submission = pd.read_csv(path +"sample_submission.csv")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Origin Model(catboost)

- OOF Ensemble



In [6]:
n_est = 3000
seed = 42
n_fold = 10
n_class = 1

X = X_train.copy()
y = y_train

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() >= 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(eval_metric="F1", task_type='GPU')
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')

In [8]:
submission['target'] = pred_test
submission

,id,target
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1


In [9]:
submission.to_csv(path + "prediction_0123_2.csv", index=False)

# Model 2(catboost)

- 하이퍼 파라미터 적용, k =10




In [ ]:
import optuna

In [ ]:
def objective(trial, data=X, target=y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 16),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.02, 0.05, 0.08, 0.1]),
        'n_estimators': trial.suggest_int('n_estimators', 2000, 8000),
        'max_bin': trial.suggest_int('max_bin', 200, 400),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log = True),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'random_seed': 42,
        'task_type': 'GPU',
        'loss_function': 'Logloss',
        'eval_metric': 'F1',
        'bootstrap_type': 'Poisson'
    }
    
    model = CatBoostClassifier(**params)  
    model.fit(X_train, y_train, eval_set = [(X_val,y_val)], early_stopping_rounds = 222, verbose = False)
    cat_pred = model.predict_proba(X_val)[:,1]
    y_pred = np.where(cat_pred >= 0.4 , 1, 0)
    F1_score = f1_score(y_val, y_pred)

    return F1_score

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

Number of finished trials: 30

Best trial: {'max_depth': 10, 'learning_rate': 0.02, 'n_estimators': 2099, 'max_bin': 231, 'min_data_in_leaf': 61, 'l2_leaf_reg': 0.21933100943955486, 'subsample': 0.6525918984479465}

Best value: 0.6913040754315071

In [ ]:
seed = 42
n_fold = 7
n_class = 1

X = X_train.copy()
y = y_train

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() >= 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(max_depth=10,learning_rate=0.02,n_estimators=2099,max_bin=231,min_data_in_leaf=61,l2_leaf_reg=0.2193,subsample=0.6525,eval_metric="F1", task_type='GPU',bootstrap_type='Poisson')
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')

##### 1. 현재 모델 성능 보기 
- (1) 제출 : 0.697899159
- (2) valid: 0.708

##### 2. K-fold 앙상블 적용하지 않고, 모델 생성 후 점수 보기
##### 3. fold 개수 변경해보기 
- (1) k=15 => 제출: 0.6969100675	

##### 4. 피처 인포턴스 기반으로 피처 개수 정리
- 5개 정도 피처 제거 해보고 나머지 다른거

##### 5. one-hot-max 파라미터 수정
(1) 
- one_hot_max_size = 20
- k=7
- optuna 최적 하이퍼 파라미터 적용

##### 6. 결정트리 스태킹
##### 7. 하이퍼 파라미터 적용
- optuna 최적 하이퍼 파라미터 적용 
- valid :0.71 제출: 0.6843


#Model 3 one_hot_max_size = 20

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() > 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(max_depth=10,learning_rate=0.02,n_estimators=2099,max_bin=231,min_data_in_leaf=61,l2_leaf_reg=0.2193,subsample=0.6525,eval_metric="F1", task_type='GPU',bootstrap_type='Poisson', one_hot_max_size=20)
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')



```
----------------- Fold 0 -----------------

0:	learn: 0.6251861	test: 0.6263958	best: 0.6263958 (0)	total: 698ms	remaining: 24m 25s
100:	learn: 0.6475928	test: 0.6477361	best: 0.6477361 (100)	total: 57s	remaining: 18m 46s
200:	learn: 0.6596423	test: 0.6606113	best: 0.6608103 (198)	total: 1m 53s	remaining: 17m 50s
300:	learn: 0.6691474	test: 0.6678405	best: 0.6679603 (295)	total: 2m 50s	remaining: 16m 55s
400:	learn: 0.6771361	test: 0.6706509	best: 0.6707179 (375)	total: 3m 45s	remaining: 15m 55s
500:	learn: 0.6841943	test: 0.6723041	best: 0.6723131 (499)	total: 4m 41s	remaining: 14m 57s
600:	learn: 0.6909437	test: 0.6734091	best: 0.6737398 (593)	total: 5m 34s	remaining: 13m 53s
700:	learn: 0.6973506	test: 0.6741371	best: 0.6742679 (686)	total: 6m 26s	remaining: 12m 51s
800:	learn: 0.7036685	test: 0.6740731	best: 0.6747530 (730)	total: 7m 19s	remaining: 11m 51s
bestTest = 0.6747529795
bestIteration = 730
Shrink model to first 731 iterations.
CV F1 score: 0.705351

----------------- Fold 1 -----------------

0:	learn: 0.6319002	test: 0.6334199	best: 0.6334199 (0)	total: 607ms	remaining: 21m 14s
100:	learn: 0.6485649	test: 0.6512819	best: 0.6512998 (98)	total: 56.5s	remaining: 18m 37s
200:	learn: 0.6607434	test: 0.6639592	best: 0.6639858 (198)	total: 1m 52s	remaining: 17m 39s
300:	learn: 0.6692984	test: 0.6679811	best: 0.6683066 (298)	total: 2m 48s	remaining: 16m 44s
400:	learn: 0.6778644	test: 0.6726720	best: 0.6728206 (397)	total: 3m 42s	remaining: 15m 41s
500:	learn: 0.6850085	test: 0.6740020	best: 0.6741150 (497)	total: 4m 39s	remaining: 14m 50s
600:	learn: 0.6916299	test: 0.6750486	best: 0.6752041 (593)	total: 5m 33s	remaining: 13m 52s
700:	learn: 0.6980000	test: 0.6766882	best: 0.6767342 (688)	total: 6m 27s	remaining: 12m 53s
800:	learn: 0.7038875	test: 0.6771395	best: 0.6772380 (795)	total: 7m 20s	remaining: 11m 54s
900:	learn: 0.7099811	test: 0.6775234	best: 0.6777386 (895)	total: 8m 13s	remaining: 10m 56s
1000:	learn: 0.7153245	test: 0.6779229	best: 0.6782558 (959)	total: 9m 4s	remaining: 9m 56s
1100:	learn: 0.7201424	test: 0.6782979	best: 0.6788506 (1063)	total: 9m 55s	remaining: 8m 59s
bestTest = 0.6788505503
bestIteration = 1063
Shrink model to first 1064 iterations.
CV F1 score: 0.707609

----------------- Fold 2 -----------------

0:	learn: 0.6326581	test: 0.6266336	best: 0.6266336 (0)	total: 695ms	remaining: 24m 17s
100:	learn: 0.6490684	test: 0.6433576	best: 0.6434332 (99)	total: 55.8s	remaining: 18m 23s
200:	learn: 0.6606139	test: 0.6548769	best: 0.6548815 (194)	total: 1m 52s	remaining: 17m 40s
300:	learn: 0.6706686	test: 0.6621457	best: 0.6621457 (300)	total: 2m 48s	remaining: 16m 45s
400:	learn: 0.6791609	test: 0.6647418	best: 0.6647418 (400)	total: 3m 43s	remaining: 15m 47s
500:	learn: 0.6867236	test: 0.6676882	best: 0.6677142 (499)	total: 4m 38s	remaining: 14m 48s
600:	learn: 0.6929215	test: 0.6697923	best: 0.6698804 (594)	total: 5m 31s	remaining: 13m 46s
700:	learn: 0.6992501	test: 0.6695477	best: 0.6704390 (619)	total: 6m 22s	remaining: 12m 43s
bestTest = 0.6704390347
bestIteration = 619
Shrink model to first 620 iterations.
CV F1 score: 0.702465

----------------- Fold 3 -----------------

0:	learn: 0.6255184	test: 0.6238619	best: 0.6238619 (0)	total: 723ms	remaining: 25m 17s
100:	learn: 0.6485452	test: 0.6506126	best: 0.6508597 (95)	total: 56.2s	remaining: 18m 31s
200:	learn: 0.6604876	test: 0.6640951	best: 0.6641641 (199)	total: 1m 52s	remaining: 17m 45s
300:	learn: 0.6701569	test: 0.6696937	best: 0.6696937 (300)	total: 2m 48s	remaining: 16m 47s
400:	learn: 0.6782281	test: 0.6715659	best: 0.6715659 (400)	total: 3m 44s	remaining: 15m 48s
500:	learn: 0.6853787	test: 0.6732039	best: 0.6733233 (499)	total: 4m 39s	remaining: 14m 50s
600:	learn: 0.6915586	test: 0.6749101	best: 0.6750558 (599)	total: 5m 32s	remaining: 13m 49s
700:	learn: 0.6984827	test: 0.6757107	best: 0.6757107 (700)	total: 6m 24s	remaining: 12m 47s
800:	learn: 0.7048791	test: 0.6771579	best: 0.6773301 (794)	total: 7m 14s	remaining: 11m 44s
900:	learn: 0.7104119	test: 0.6768007	best: 0.6775942 (831)	total: 8m 6s	remaining: 10m 47s
bestTest = 0.6775941872
bestIteration = 831
Shrink model to first 832 iterations.
CV F1 score: 0.706319

----------------- Fold 4 -----------------

0:	learn: 0.6245210	test: 0.6275553	best: 0.6275553 (0)	total: 614ms	remaining: 21m 27s
100:	learn: 0.6464366	test: 0.6471011	best: 0.6472435 (99)	total: 55.7s	remaining: 18m 21s
200:	learn: 0.6588890	test: 0.6612023	best: 0.6612023 (200)	total: 1m 51s	remaining: 17m 35s
300:	learn: 0.6689588	test: 0.6686738	best: 0.6686738 (299)	total: 2m 46s	remaining: 16m 36s
400:	learn: 0.6776242	test: 0.6710346	best: 0.6711548 (398)	total: 3m 43s	remaining: 15m 46s
500:	learn: 0.6848524	test: 0.6724523	best: 0.6725723 (476)	total: 4m 38s	remaining: 14m 48s
600:	learn: 0.6922577	test: 0.6745712	best: 0.6747007 (595)	total: 5m 32s	remaining: 13m 47s
700:	learn: 0.6989838	test: 0.6752917	best: 0.6753682 (695)	total: 6m 25s	remaining: 12m 48s
800:	learn: 0.7053536	test: 0.6762103	best: 0.6762103 (800)	total: 7m 18s	remaining: 11m 50s
900:	learn: 0.7105887	test: 0.6771964	best: 0.6774044 (893)	total: 8m 10s	remaining: 10m 51s
1000:	learn: 0.7158363	test: 0.6770096	best: 0.6775538 (926)	total: 9m 1s	remaining: 9m 53s
bestTest = 0.6775538293
bestIteration = 926
Shrink model to first 927 iterations.
CV F1 score: 0.707595

----------------- Fold 5 -----------------

0:	learn: 0.6198051	test: 0.6262259	best: 0.6262259 (0)	total: 662ms	remaining: 23m 7s
100:	learn: 0.6478593	test: 0.6447282	best: 0.6449458 (98)	total: 57.1s	remaining: 18m 48s
200:	learn: 0.6602349	test: 0.6591299	best: 0.6594166 (197)	total: 1m 52s	remaining: 17m 38s
300:	learn: 0.6695311	test: 0.6661682	best: 0.6663660 (296)	total: 2m 49s	remaining: 16m 49s
400:	learn: 0.6774738	test: 0.6685936	best: 0.6686973 (393)	total: 3m 44s	remaining: 15m 50s
500:	learn: 0.6850610	test: 0.6709758	best: 0.6710264 (496)	total: 4m 38s	remaining: 14m 47s
600:	learn: 0.6922163	test: 0.6724063	best: 0.6724063 (600)	total: 5m 31s	remaining: 13m 46s
700:	learn: 0.6985033	test: 0.6739953	best: 0.6739953 (700)	total: 6m 24s	remaining: 12m 45s
800:	learn: 0.7044885	test: 0.6752068	best: 0.6755404 (772)	total: 7m 16s	remaining: 11m 47s
bestTest = 0.6755404138
bestIteration = 772
Shrink model to first 773 iterations.
CV F1 score: 0.705002

----------------- Fold 6 -----------------

0:	learn: 0.6293640	test: 0.6361385	best: 0.6361385 (0)	total: 672ms	remaining: 23m 29s
100:	learn: 0.6479754	test: 0.6540539	best: 0.6543276 (93)	total: 56s	remaining: 18m 28s
200:	learn: 0.6583306	test: 0.6655087	best: 0.6656023 (199)	total: 1m 52s	remaining: 17m 42s
300:	learn: 0.6682678	test: 0.6714277	best: 0.6714949 (298)	total: 2m 48s	remaining: 16m 48s
400:	learn: 0.6769656	test: 0.6749085	best: 0.6749085 (400)	total: 3m 43s	remaining: 15m 46s
500:	learn: 0.6846385	test: 0.6770406	best: 0.6770406 (500)	total: 4m 37s	remaining: 14m 43s
600:	learn: 0.6914008	test: 0.6786257	best: 0.6786257 (600)	total: 5m 31s	remaining: 13m 46s
700:	learn: 0.6976550	test: 0.6798427	best: 0.6798890 (697)	total: 6m 24s	remaining: 12m 47s
800:	learn: 0.7041230	test: 0.6798000	best: 0.6799068 (703)	total: 7m 17s	remaining: 11m 48s
900:	learn: 0.7096360	test: 0.6797272	best: 0.6802553 (881)	total: 8m 8s	remaining: 10m 49s
1000:	learn: 0.7148863	test: 0.6798017	best: 0.6803971 (935)	total: 9m	remaining: 9m 53s
bestTest = 0.6803971467
bestIteration = 935
Shrink model to first 936 iterations.
CV F1 score: 0.708821
	F1 score: 0.706159
```



# Model 4 one_hot_max_size =10

In [ ]:
#k 개의  모델을 다 사용하되 앙상블 처럼 각 모델에서 주는 output 들의 평균값을 사용하여 최종 pred 산출(OOF Ensemble)

skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = 0
cat_cols = X_train.columns[X_train.nunique() > 2].tolist()
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(max_depth=10,learning_rate=0.02,n_estimators=2099,max_bin=231,min_data_in_leaf=61,l2_leaf_reg=0.2193,subsample=0.6525,eval_metric="F1", task_type='GPU',bootstrap_type='Poisson', one_hot_max_size=10)
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = pd.DataFrame(model_cat.predict_proba(X_valid)[:, 1], index=X_valid.index)
  cat_pred_test = cat_pred_test + (model_cat.predict_proba(X_test)[:, 1] / n_fold)
  pred = np.where(cat_pred >= 0.4 , 1, 0)
  pred_test = np.where(cat_pred_test >= 0.4 , 1, 0)

  print(f'CV F1 score: {f1_score(y_valid, pred[valid_idx]):.6f}')
    
print(f'\tF1 score: {f1_score(y, pred):.6f}')